### BreachSeek Redo

<img src="https://arxiv.org/html/2409.03789v1/extracted/5823759/Graph.png" width=500px />

In [ ]:
# utils/model.py
from langchain_openai import ChatOpenAI
# from langchain_anthropic import ChatAnthropic

def _get_model(model_name: str = "openai"):
    # if model_name == "anthropic":
    #     model = ChatAnthropic(temperature=0, model_name="claude-3-5-sonnet-20240620")
    if model_name == "openai":
        model = ChatOpenAI(temperature=0, model_name="gpt-4")
    else:
        raise ValueError(f"Unsupported model type: {model_name}")

    return model

In [ ]:
# utils/tools.py
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import ShellTool
from langgraph.prebuilt import ToolNode

tools = [DuckDuckGoSearchRun(), ShellTool()]
tool_node = ToolNode(tools)

In [ ]:
# utils/state.py
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage, BaseMessage
from typing_extensions import TypedDict, Annotated, Sequence, List,  Literal, Optional, Dict
import operator

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


class RecoderOptions(TypedDict):
    report: str
    generate_final_report: bool
    file_names: List[str]


class PentestState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], operator.add]
    current_step: Literal['pentester', 'evaluator', 'recorder', '__end__']
    pentest_results: dict
    pentest_tasks: list
    task: str 
    evaluation: str 
    model_status: str
    findings: Dict[str, List[str]]
    command: str
    tool_name: str
    tool_results: str 
    recorder_options: Optional[RecoderOptions]


class GraphConfig(TypedDict):
    model_name: Literal["anthropic", "openai"]
    pentester_model: Literal["ollama"," anthropic", "openai"]

In [ ]:
# utils/nodes.py
from langgraph.prebuilt import ToolNode


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there are no tool calls, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


system_prompt = """
Be a helpful agent, you can search the web and you can run shell commands.
If you receive the output from a tool
show the output of the commands in markdown for debugging purposes prefix your
response after the tool call with DEBUG: keyword.
"""


def call_model(state, config):
    messages = state["messages"]
    messages = [{"role": "system", "content": system_prompt}] + messages
    model_name = config.get('configurable', {}).get("model_name", "openai")
    model = _get_model(model_name)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
from langchain_core.messages import SystemMessage

# supervisor.py
supervisor_system_prompt = SystemMessage("""
You are the supervisor of AI agents responsible for overseeing their performance, 
you send the pentesting tasks to pentester agent, and you receive the response 
from an evaluator agent, if the evaluator says everything is done, then end the program,

Try to end as soon as possible. 
""")


class Report(TypedDict):
    report: Annotated[str, "This is the summary written in Latex"]
    file_name: Annotated[str, "This is the filename of the Latex report"]


# Don't think we need this one
def _swap_messages(messages):
    new_messages = []
    for m in messages:
        if m['role'] == 'assistant':
            new_messages.append({"role": "user", "content": m['content']})
        else:
            new_messages.append({"role": "assistant", "content": m['content']})
    return new_messages


def supervisor(state: PentestState) -> PentestState:
    # If all evaluator concludes all tasks are done -> Termination
    if state['evaluation'] and 'end' in state['evaluation']:
        return {
                'messages': [{'role': 'assistant', 'content': 'done, bye!'}],
                'current_step': '__end__',
                }


    messages = [system_prompt] + state['messages']

    # Pentester ran, and evaluation has been made (Yet done)
    if state['evaluation']:
        messages = [system_prompt] + [HumanMessage(content=f"based on {state['evaluation']}, what should we do now to finish the tasks: {state['pentest_tasks']}")]
    # messages = [system_prompt] + state['messages']
    model = _get_model().with_structured_output(SupervisorTask)
    response = model.invoke(messages) # error here
    # If supervisor concludes all tasks are done -> Termination
    if response['done']:
        return {
                'messages': [{'role': 'assistant', 'content': 'done, bye!'}],
                'current_step': '__end__',
                }

    # In case the pentester should be run
    state['current_step'] = response['next_agent']
    if response['next_agent'] == 'pentester':
        return {
                'messages': [{'role': 'assistant', 
                             'content': response['supervisor_thought']}],
                'current_step': 'pentester',
                'pentest_tasks': response['tasks'],
                }

    # Base condition (?)
    return {
                'messages': [{'role': 'assistant', 'content': 'done, bye!'}],
                'current_step': '__end__',
            }

In [19]:
# recorder.py
class Report(TypedDict):
    report: Annotated[str, "This is the summary written in Latex"]
    file_name: Annotated[str, "This is the filename of the Latex report"]



def recorder_summary(state: PentestState, model):
    prompt = '''You are tasked with recording and summarizing information of a chat between a human and an LLM in \
    which the LLM is utilizing the command line to execute tasks. You have as an input the history of the last command that ran \
    which inclueds the output logs and the message prompt for previous commands:
     
    <history>
    {history}
    </history>

    Generate a summary for this using latex.
    '''
    formats = {
            'history': {'user_prompts': state['messages']} | state['pentest_results'],
        }
    prompt = prompt.format(**formats)

    response = model.invoke(prompt)

    state['recorder_options']['file_names'].append(response.file_name)
    
    with open(response.file_name, 'w') as f:
        f.write(response.report)

    return state

def recorder_final(state: PentestState, model):
    prompt = '''You are tasked with summarizing and reporting information of a chat between a human and an LLM in \
    whihc the LLM is utilizing the command line to execute tasks. You have as input the history of summaries of all previous \
    outputs and interactions between the human and the LLM:

    <history>
    {history}
    </history>

    You are tasked with generating a final report in a Latex format. Also return the file path as a relative path.
    '''

    summaries = []
    file_names = state['recorder_options']['file_names']
    for file_name in file_names:
        with open(file_name) as f:
            summary = f.read()
            summaries.append(summary)

    formats = {
            'history': summaries
        }
    prompt = prompt.format(**formats)

    response = model.invoke(prompt)
    state['recorder_options']['report'] = response.report
    with open(response.file_name, 'w') as f:
        f.write(response.report)

    return state



def recorder(state: PentestState):
    model = _get_model().with_structured_output(Report)
    generate_final_report = state['recorder_options']['generate_final_report']

    if generate_final_report:
        return recorder_final(state, model)
    else:
        state = recorder_summary(state, model)
        return recorder_final(state, model)

In [22]:
# pentester.py
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from typing_extensions import Annotated, TypedDict, List, Literal, Optional 


class PentesterResults(TypedDict):
    """The results that the pentester needs to work"""
    message: Annotated[str, ..., "Pentester thought process"]
    phase: Annotated[str, ..., "Current phase of pentesting, such as scanning or exploitation"]
    tasks: Annotated[List[str], ..., "Tasks to perform to complete the current phase"]
    results: Annotated[List[str], ..., "Summary of results"]
    tool_use: Annotated[bool, ..., "Indicates if tools are still needed to complete a task"]
    software: Annotated[List[str], ..., "program names that will be used to perform the tasks"]
    command: Annotated[str, ..., "possible command that will be run"]


def _swap_messages(messages):
    new_messages = []
    for m in messages:
        if m['role'] == 'assistant':
            new_messages.append({"role": "user", "content": m['content']})
        else:
            new_messages.append({"role": "assistant", "content": m['content']})
    return new_messages


def pentester(state: PentestState) -> PentestState:
    # print(state)
    tasks = f"""
    Do the following tasks:

    {state['pentest_tasks']}
    """

    if state['tool_results']: # remove later, for debugging
        return {'current_step': 'evaluator'}

    # messages = [system_prompt, HumanMessage(content=tasks)] 
    messages = [system_prompt] + [{'role': 'user', 'content': tasks}] + _swap_messages(state['messages'])
    model = _get_model().with_structured_output(PentesterResults)
    response = model.invoke(messages)
    print(f'pentester:\n{response = }')

    # In case that the tool needs to be used
    if response['tool_use']:
        return {
                'messages': [{'role': 'assistant',
                              'content': response['message']}],
                              # 'tool_calls': response['software']}], 
                'current_step': 'pentester',
                'tool_name': response['software'][0],
                'task': response['tasks'][0],
                # "command": "nmap -sV -sC -p- -oN metasploitable2_scan.txt 192.168.100.231",
                "command": response['command'],
                }
    return {'current_step': 'evaluator'}

In [ ]:
# tools.py
class ToolUsage(TypedDict):
    """The task that the tool needs to do"""
    message: str
    # phase: Annotated[str, ... , "Current phase of pentesting"]
    # task: Annotated[str, ... , "Task to perform so that the tool agent succeeds."]
    # hints: Annotated[Optional[List[str]], ... , "May contain hints about the task"]
    # constraints: Annotated[Optional[List[str]], ... , "Contains constraints that the tool user agent should be aware of"]
    # program_name: Annotated[str, ... , "Program name that the tool uses to finish a task"]
    # args: Annotated[Optional[Dict], ... , "Args that might help the tool user agent"]
    # results: Annotated[Dict, ... , "Title of findings as keys, and a description as their value"]
    results: str


def tools_node(state: PentestState) -> PentestState:
    system_prompt = SystemMessage(f"""
    You are an AI agent who is an expert in using a tool named {state['tool_name']}, 
    to do {state['pentest_tasks']}. You are working with a team of AI agents, 
    you will receive a task, and possibly hints, constrains, and args. 
    You will have to use your expertise in {state['tool_name']} to finish
    the task. When you are done pass the results to the calling agent. 

    you have access to bash shell using shell_tool
    """)
    print(f'{system_prompt = }')
    human_msg = HumanMessage(content=f"Use {state['tool_name']} to do {state['task']}")
    messages = [system_prompt] + [human_msg]
    model = _get_model().bind_tools(tools)
    ai_msg = model.invoke(messages)
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {tool.name: tool for tool in tools}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        # tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)
    # tool_msg = shell_tool.invoke(ai_msg.tool_calls)
    # messages.append(tool_msg)
    messages.append(HumanMessage(content='summarize before parsing'))
    model = _get_model().with_structured_output(ToolUsage)
    response = model.invoke(messages)
    print(f'tools:\n{response = }')
    return {
            'messages': [{'role': 'assistant',
                          'content': '\n\n'.join(response.values())}],
            'tool_results': response['results'],
            }